<style>
.custom-font {
    font-family: "Courier New", serif;
    font-size: 18px;
}
</style>

# CUDA Quantum 101
    Important Links 
    
    * Installation 
        https://nvidia.github.io/cuda-quantum/latest/install.html 
    * Documentation
        https://nvidia.github.io/cuda-quantum/latest/index.html
    * CUDA Quantum repo
        https://github.com/NVIDIA/cuda-quantum

    * CUDA Quantum kernels 
        https://nvidia.github.io/cuda-quantum/latest/using/cudaq/kernel.html 
    * Algorithmic primitives
        https://nvidia.github.io/cuda-quantum/latest/index.html
    * Targets
        https://github.com/NVIDIA/cuda-quantum


    Outline 
    
    1. What is CUDA Quantum? 

    2. CUDA Quantum Kernels
        2.1 Simple kernels
        2.2 Adjoint 
        2.3 Conditionals 

    3. Algorithmic primitives
        3.1 cudaq.sample() 
        3.2 cudaq.spin_op()
        3.3 cudaq.observe()

    4. Parameterized circuits in CUDA Quantum
        4.1 VQA with cudaq.observe() and cudaq.optimize() 
        4.2 VQA with cudaq.vqe() wrapper

    5. Targets
        5.1 Quantum Hardware Integration
        5.2 Quantum Circuit Simulation
            5.2.1 cuQuantum Simulator backends
                    - Statevector simulator (custatevec)
                    - Tensornet simulator (cutensornet)
            5.2.2 Density-matrix simulator (dm)
            
    6. Noise modeling 
        6.1 Background 
        6.2 Simple examples
            - Bitflip error
            - Custom noise model 
        
    7. Scalability 101 

 ### 1. What is CUDA Quantum?

    
    * An open-source platform for integrating and programming quantum processing 
      units (QPUs), GPUS and CPUs in one system.

    * Enables dynamic workflow between different system architectures. 

    * A programming model which extends C++ and Python with quantum kernels, enabling 
      high-level programming in familiar languages.

    * The ability to utilize and seamlessly switch between different quantum technologies, 
      including state-of-the-art simulator backends with NVIDIA cuQuantum and a number of 
      different physical quantum processors (QPUs). 

In [ ]:
# Import the CUDA Quantum module
import cudaq

### 2. CUDA Quantum Kernels

    2.1 Simple kernel

In [ ]:
# We begin by defining the `Kernel` that we will construct our
# program with.
kernel = cudaq.make_kernel()

In [ ]:
# Next, we can allocate qubits to the kernel via `qalloc(qubit_count)`.
# An empty call to `qalloc` will return a single qubit.
qubit = kernel.qalloc()

In [ ]:
# Now we can begin adding instructions to apply to this qubit!
# Here we'll just add every non-parameterized
# single qubit gates that are supported by CUDA Quantum.
kernel.h(qubit)
kernel.x(qubit)
kernel.y(qubit)
kernel.z(qubit)
kernel.t(qubit)
kernel.s(qubit)

In [ ]:
# Next, we add a measurement to the kernel so that we can sample
# the measurement results on our simulator!
kernel.mz(qubit)

In [ ]:
# To list other methods and attributes available to the kernel object
#dir(kernel)
#help(kernel.tdg)

    2.1 Adjoint of a Kernel

In [ ]:
import cudaq 

# Create a kernel and do some operations
other_kernel = cudaq.make_kernel()
other_qubit = other_kernel.qalloc()
other_kernel.x(other_qubit)

# Create a kernel, which'll be the adjoint of other_kernel 
kernel = cudaq.make_kernel()
kernel.adjoint(other_kernel)

      2.2 Conditional Measurement

In [ ]:
# The conditional measurement functionality of `cudaq.kernel`
import cudaq 

kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

def then_function():
    kernel.x(qubit)

kernel.x(qubit)

# Measure the qubit.
measurement_ = kernel.mz(qubit)
# Apply `then_function` to the `kernel` if
# the qubit was measured in the 1-state.
#kernel.c_if(measurement_, then_function)

# Measure the qubit again.
result = cudaq.sample(kernel, shots_count=30) # adjust number of shots
result.dump()

#assert len(result) == 1
# Qubit should be in the 0-state after undergoing
# two X rotations.
#assert '0' in result

###     3. Algorithmic primitives

    Algorithmic primitives are common programming patterns that have been implemented in 
    the CUDA Quantum library.

    3.1 cudaq.sample()
    3.2 cudaq.observe()
    3.3 cudaq.spin_op()

#### 3.1. cudaq.sample()

   
    The sample() function performs multiple measurements of the circuit(1000 shots by default) 
    and returns a dictionary of the measurement outcomes along with their respective counts. 


In [ ]:
# Finally, we can execute this kernel on the state vector simulator
# by calling `cudaq.sample`. This will execute the provided kernel
# `shots_count` number of times and return the sampled distribution
# as a `cudaq.SampleResult` dictionary.
sample_result = cudaq.sample(kernel)

# Now let's take a look at the `SampleResult` we've gotten back!
print(sample_result)  # or result.dump()     

    Putting it all together!

In [ ]:
import cudaq

kernel = cudaq.make_kernel()
qubit = kernel.qalloc(2)
                                  
kernel.h(qubit)
kernel.x(qubit)
kernel.y(qubit)
kernel.z(qubit)
kernel.t(qubit)
kernel.s(qubit)

kernel.mz(qubit)

# 1000 is the default
sample_result = cudaq.sample(kernel, shots_count=2000) 

print(sample_result)  # or sample_result.dump()  

In [ ]:
# To list other methods and attributes available to the kernel object
#dir(sample_result)
#help(sample_result.values)

In [ ]:
# Extracting data from sample

print(f"most probable = {sample_result.most_probable()}")
print(f"expectation_value = {sample_result.expectation_z()}")
print(f"count = {sample_result.count('1')}")
print(f"probability = {sample_result.probability('1')}")


In [ ]:
# clear results, result should now be empty
sample_result.clear()

####  3.2. cudaq.spin_op()

     
    The spin_op represents a general sum of Pauli tensor products. It exposes the typical 
    algebraic operations that allow programmers to define primitive Pauli operators and use
    them to compose larger, more complex Pauli tensor products and sums thereof. 

    Let's say our Hamitonian is $Z_0 \otimes I_1 + I_0 \otimes X_1 + Y_0 \otimes I_1 + Y_0 \otimes Y_1$.

In [ ]:
# Importing the spin_op
from cudaq import spin

# the obseravle 
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0) + spin.y(0)*spin.y(1)
hamiltonian.dump()

In [ ]:
#dir(hamiltonian)
#help(hamiltonian.dump)

#### 3.3. cudaq.observe()

 
    Compute the expected value of the observable.

In [ ]:
# First we need to construct a cuda quantum kernel
kernel = cudaq.make_kernel()
qreg = kernel.qalloc(2)
kernel.x(qreg[0])

In [ ]:
# The cudaq.observe() takes the quantum circuit and the observable as input params
observe_result = cudaq.observe(kernel, hamiltonian, shots_count=1000)

In [ ]:
print(observe_result.dump())
observe_result.expectation_z()

In [ ]:
# For a complete list of attributes
# dir (observe_result)

    Putting it all together!

In [ ]:
import cudaq
from cudaq import spin

# First we need to construct a cuda quantum kernel
kernel = cudaq.make_kernel()
qreg = kernel.qalloc(2)
kernel.x(qreg[0])

# The cudaq.observe() takes the quantum circuit and the observable as input params
observe_result = cudaq.observe(kernel, hamiltonian, shots_count=10000)

print(observe_result.dump())
observe_result.expectation_z()

### 4. Parameterized circuits

In [ ]:
import cudaq
from cudaq import spin

# the obserable 
hamiltonian = 5.907 - 2.1433 * spin.x(0) * spin.x(1) \
            - 2.1433 * spin.y(0) * spin.y(1) + 0.21829 * spin.z(0) \
            - 6.125 * spin.z(1)

# parameterized cudaq kernel, the parameter is of type float
kernel, theta = cudaq.make_kernel(float)
q = kernel.qalloc(2)
kernel.x(q[0])
kernel.ry(theta, q[1])
kernel.cx(q[1], q[0])

# observe() takes the kernel, the observable and the kernel paramter(s)
# as args
observe_result = cudaq.observe(kernel, hamiltonian, .59)
print(observe_result.dump())
observe_result.expectation_z()

    4.1 Variational Algorithms in CUDA Quantum

    Variational algorithms in CUDA Quantum typically leverage the `cudaq.observe(...)` 
    function in tandem with the `cudaq.optimizer`.
    One can choose an optimization strategy provided as specific sub-types of the `cudaq.optimizer`.

In [ ]:
import cudaq
from cudaq import spin

# Parameterized circuit with theta as the parameter
kernel, theta = cudaq.make_kernel(list)
qreg = kernel.qalloc(2)
kernel.x(qreg[0])
kernel.ry(theta[0], qreg[1])


# Observable  
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0)

# Initialize the gradient-free optimizer COBYLA
optimizer = cudaq.optimizers.COBYLA()

# Specify the number of iterations (optional)
optimizer.max_iterations = 5

def cost_function(x):
    # cudaq.observe() produces the expected value of a specified observable wrt
    # a given parameterized ansatz at given params.
    # This value is the cost function wrt which we are optimizing.
    observeResult = cudaq.observe(kernel, hamiltonian, x)
    print (observeResult.expectation_z(), x)
    return observeResult.expectation_z()

# Carry out the optimization
opt_value, opt_theta = optimizer.optimize(dimensions=1, function=cost_function)

      4.2 VQE wrapper

In [ ]:
# Import the necessary modules
import cudaq
from cudaq import spin

# Parameterized circuit with theta as the parameter
kernel, theta = cudaq.make_kernel(list)
qreg = kernel.qalloc(2)
kernel.x(qreg[0])
kernel.ry(theta[0], qreg[1])

# Hamiltonian operator 
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0)

# Initialize the gradient-free optimizer COBYLA
optimizer = cudaq.optimizers.COBYLA()

# Specify the number of iterations (optional)
optimizer.max_iterations = 5

# Carry out the optimization
opt_value, opt_theta = cudaq.vqe(kernel=kernel, 
                        spin_operator=hamiltonian,
                        optimizer=optimizer,
                        parameter_count=1)

print(f"\nminimized <H> = {round(opt_value,16)}")
print(f"optimal theta = {round(opt_theta[0],16)}")

### 5. CUDA Quantum `target` 

     A `target` is a specification of the desired platform and simulator / QPU. It can be 
     specified as a runtime flag in Python. Alteratively, it can also be specified within 
     the application code. 

     Simulation backends
    - state-vector (`cuStateVec`) 
    - tensor-network (`cuTensorNet`)
    - density-matrix (`dm`) 

     Hardware support
    - CPU only, multi-threaded   
    - Single GPU   
    - Multi-GPU 
    - Multi-QPU 
    - QPU 

In [ ]:
# Print all the available targets on your machine
import cudaq

targets = cudaq.get_targets()

for t in targets:
     print(t)

#### 5.1 Quantum Hardware Integration

In [ ]:
# This code will give an error!!!!!
import cudaq

# Set the target 
cudaq.set_target("quantinuum")

# Create the kernel we'd like to execute on Quantinuum.
kernel = cudaq.make_kernel()
qubits = kernel.qalloc(2)
kernel.h(qubits[0])
kernel.cx(qubits[0], qubits[1])
kernel.mz(qubits)

# Submit to Quantinuum's endpoint and confirm the program is valid.

# By using the synchronous `cudaq.sample`, the execution of
# any remaining classical code in the file will occur only
# after the job has been executed by the Quantinuum service.
# We will use the synchronous call to submit to the syntax
# checker to confirm the validity of the program.
counts = cudaq.sample(kernel)
counts.dump()
assert (len(counts) == 2)
assert ('00' in counts)
assert ('11' in counts)

### 5.2 Simulator backends 

    cuQuantum is CUDA Quantum's workhorse for quantum circuit simulation. 
    It is a high performance library containing the following two types of simulators.

##### Statevector simulator    
    * State vector simulators serve as the main vehicle for circuit simulations. 
    * They maintain an accurate representation of the quantum state, known as the 
      state vector, throughout the simulation. 
    * Each gate that is applied corresponds to a matrix-vector multiplication.

##### Tensornet simulator 
    * The tensor network method is a technique that represents the quantum state 
      of N qubits as a series of tensor contractions.
    * The main challenge is to compute these tensor contractions efficiently. 
    * It can handle a large number of qubits for short circut depths.

    Note: To run with the cutensornet target, you will need to pull the CUDA Quantum
           docker image with the tag latest-hpc.

##### Density matrix simulator
    * Simulates quantum circuits under the influence of noise. 
    * Currently, it calls the QPP library under the hood and has only CPU support.

    To discuss the density matrix simulator further, we need to introduce a couple of new concepts.

Density matrix

    The wavefunction or state vector gives a complete description of the quantum state of an isolated quantum system. 
$\ket{\psi}$ --> ket notation for the quantum state represented as a vector


    The density matrix representation is a more general representation that is used to describe noisy quantum evolution and decoherence. It can be used to describe the pure states as well as the mixed states, which are a statistical ensemble of the pure states.

    In the density matrix notation, a pure state is given by

\begin{equation*}
\rho = \ket{\psi} \bra{\psi}.
\end{equation*}

    A mixed state is repesentated as
    
\begin{equation*}
\rho = \sum_{j} p_j \ket{\psi_j} \bra{\psi_j}, 
\end{equation*}
    where the coefficients $p_j$'s are the probabilities associated with each of the states in the ensemble.


##### Kraus Representation

    Various types of noise can be represnted mathematically using the Kraus operators.
    
\begin{equation*}
\rho \mapsto {\cal{N}}(\rho) = \sum_{j} K_j \rho K_j^{\dag}
\end{equation*}

    with the condition that 
    
\begin{equation*}
\sum_{j} K_j K_j^{\dag} = \mathbb{I}.
\end{equation*}

   Bit-flip channel
   
    - The state of the qubit is chaged from |0⟩ to |1⟩ or vice-versa
    - key-operator is Pauli X
    - Kraus reprenetation 

\begin{equation*}
    \rho = (1-p) \rho + p X\rho X 
\end{equation*}
    with the probability p in [0,1].

In [ ]:
import cudaq

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')

# CUDA Quantum supports several different models of noise. In this case,
# we will examine the modeling of decoherence of the qubit state. This
# will occur from "bit flip" errors, wherein the qubit has a user-specified
# probability of undergoing an X-180 rotation.

# We will begin by defining an empty noise model that we will add
# these decoherence channels to.
noise = cudaq.NoiseModel()

# Bit flip channel with `1.0` probability of the qubit flipping 180 degrees.
bit_flip = cudaq.BitFlipChannel(1.0)
# We will apply this channel to any X gate on the qubit, giving each X-gate
# a probability of `1.0` of undergoing an extra X-gate.
noise.add_channel('x', [0], bit_flip)

# Now we may define our simple kernel function and allocate a register
# of qubits to it.
kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

# Apply an X-gate to the qubit.
# It will remain in the |1> state with a probability of `1 - p = 0.0`.
kernel.x(qubit)
# Measure.
kernel.mz(qubit)

# Now we're ready to run the noisy simulation of our kernel.
# Note: We must pass the noise model to sample via key-word.
noisy_result = cudaq.sample(kernel, noise_model=noise)
noisy_result.dump()

# Our results should show all measurements in the |0> state, indicating
# that the noise has successfully impacted the system.

# To confirm this, we can run the simulation again without noise.
# We should now see the qubit in the |1> state.
noiseless_result = cudaq.sample(kernel)
noiseless_result.dump()

Custom Noise Model

     Here, we demonstrate a custom noise model with the same Kraus operators as in the ampltiude damping channel, but following the same template we can build other noise models such as the Pauli noise model.
     

In [ ]:
import cudaq
import numpy as np

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')

# CUDA Quantum supports custom noise models through the definition of
# `KrausChannel`'s. In this case, we will define a set of `KrausOperator`'s
# that  affect the same noise as the `AmplitudeDampingChannel`. This
# channel will model the energy dissipation within our system via
# environmental interactions. With a variable probability, it will
# return the qubit to the |0> state.

# We will begin by defining an empty noise model that we will add
# our Kraus Channel to.
noise = cudaq.NoiseModel()


# We will define our Kraus Operators within functions, as to
# allow for easy control over the noise probability.
def kraus_operators(probability):
    """See Nielsen, Chuang Chapter 8.3.5 for definition source."""
    kraus_0 = np.array([[1, 0], [0, np.sqrt(1 - probability)]],
                       dtype=np.complex128)
    kraus_1 = np.array([[0, 0], [np.sqrt(probability), 0]], dtype=np.complex128)
    return [kraus_0, kraus_1]


# Manually defined amplitude damping channel with `1.0` probability
# of the qubit decaying to the ground state.
amplitude_damping = cudaq.KrausChannel(kraus_operators(1.0))
# We will apply this channel to any Hadamard gate on the qubit.
# Meaning, after each Hadamard on the qubit, there will be a
# probability of `1.0` that the qubit decays back to ground.
noise.add_channel('h', [0], amplitude_damping)

# Now we may define our simple kernel function and allocate a qubit.
kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

# Then we apply a Hadamard gate to the qubit.
# This will bring it to `1/sqrt(2) (|0> + |1>)`, where it will remain
# with a probability of `1 - p = 0.0`.
kernel.h(qubit)

# Measure.
kernel.mz(qubit)

# Now we're ready to run the noisy simulation of our kernel.
# Note: We must pass the noise model to sample via key-word.
noisy_result = cudaq.sample(kernel, noise_model=noise)
noisy_result.dump()

# Our results should show all measurements in the |0> state, indicating
# that the noise has successfully impacted the system.

# To confirm this, we can run the simulation again without noise.
# The qubit will now have a 50/50 mix of measurements between
# |0> and |1>.
noiseless_result = cudaq.sample(kernel)
noiseless_result.dump()

###  7. Power of GPU acceleration 101

    Make sure that that your Docker container was launched with the `--gpus all` flag turned on.

    Bell State 
 $\dfrac{1}{\sqrt{2}}(\ket{00} +\ket{11})$, maximally entangled 2-qubit state.

    GHZ state
$\dfrac{1}{\sqrt{2}}(\ket{0}^{\otimes N} +\ket{1}^{\otimes N})$, maximally entangled N-qubit state, $N \ge 3$.

In [ ]:
# To run with available gpu using a command line flag:
# python ghz_state.py --target nvidia
# To run with cpu (very slow) using a command line flag:
# python ghz_state.py


import cudaq

cudaq.set_target("nvidia") # change target to "default" for cpu

def ghz_state(N):

    kernel = cudaq.make_kernel()
    q = kernel.qalloc(N)

    kernel.h(q[0])
    for i in range(N - 1):
      kernel.cx(q[i], q[i + 1])

    kernel.mz(q)
    return kernel

n = 30
print("Preparing GHZ state for", n, "qubits.")
kernel = ghz_state(n)
counts = cudaq.sample(kernel)
counts.dump()

    CPU (AMD Threadripper Pro 5965wx) vs GPU (NVIDIA RTX A6000) timing on my system from a single run: 11m 54s vs 2 seconds.